<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Extending Standard Algorithms

## Content
* [Extended Algorithm Example](#Extended-Algorithm-Example)
* [Iterators](#Iterators)
* [CUDA Fancy Iterators](#CUDA-Fancy-Iterators)
* [Exercise: Computing Variance](01.03.02-Exercise-Computing-Variance.ipynb)

---

Occasionally, you might encounter a use case that's not directly covered by standard algorithms.
In this section, we'll explore techniques that can help you extend existing algorithms to your unique use cases.


In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"): # If running in Google Colab:
  !mkdir -p Sources
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-cpp-tutorial/notebooks/01.03-Extending-Algorithms/Sources/ach.h -nv -O Sources/ach.h

## Extended Algorithm Example
Let's consider a scenario where such customization might be necessary. 
In our earlier example, we were computing the next temperature based on the previous one.
Now let's say we have to find the maximum change in temperature made in the current step.

![Max Diff](Images/max-diff.svg "Max Diff")

In the previous section, you learned how to port standard algorithms to GPU.
It's sufficient to use `thrust::` namespace for containers and algorithms as follows.

In [ ]:
%%writefile Sources/naive-max-diff.cpp
#include "ach.h"

float naive_max_change(const thrust::universal_vector<float>& a,
                       const thrust::universal_vector<float>& b)
{
    // allocate vector to store `a - b`
    thrust::universal_vector<float> unnecessarily_materialized_diff(a.size());

    // compute products
    thrust::transform(thrust::device,
                      a.begin(), a.end(),                       // first input sequence
                      b.begin(),                                // second input sequence
                      unnecessarily_materialized_diff.begin(),  // result
                      []__host__ __device__(float x, float y) { // transformation (abs diff)
                         return abs(x - y);
                      });

    // compute max difference
    return thrust::reduce(thrust::device,
                          unnecessarily_materialized_diff.begin(),
                          unnecessarily_materialized_diff.end(),
                          0.0f, thrust::maximum<float>{});
}

int main()
{
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp[] = {{ 42, 24, 50 }, { 0, 0, 0}};
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  max-change\n");
    for (int step = 0; step < 3; step++) {
        thrust::universal_vector<float> &current = temp[step % 2];
        thrust::universal_vector<float> &next = temp[(step + 1) % 2];

        thrust::transform(thrust::device, current.begin(), current.end(), next.begin(), transformation);
        std::printf("%d     %.2f\n", step, naive_max_change(current, next));
    }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/naive-max-diff.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

In the code above, we started by allocating storage for products:

```c++
thrust::universal_vector<float> unnecessarily_materialized_diff(a.size());
```

Then, we used a version of `thrust::transform` algorithm that accepts two sequences.
It then applies transformation to each pair of elements in these sequences.
In our case, the transformation is computing absolute difference:

```c++
thrust::transform(thrust::device, 
                  a.begin(), a.end(),                       // first input sequence
                  b.begin(),                                // second input sequence
                  diff.begin(),                             // result
                  []__host__ __device__(float x, float y) { // transformation (abs diff)
                      return abs(x - y); 
                  });
```

And finally, we use `thrust::reduce` with the `thrust::maximum` operator to find the maximum absolute difference:

```c++
    return thrust::reduce(thrust::device, 
                          unnecessarily_materialized_diff.begin(), 
                          unnecessarily_materialized_diff.end(), 
                          0.0f, thrust::maximum<float>{});
```

Although this implementation is functionally correct, it's far from being performant.
To discover this inefficiency, let's consider our algorithm from the memory point of view.
Let's count all memory accesses in our naive implementation.
As part of the transformation step, 
we read `2 * n` floats total from `a` and `b` and store `n` elements back to memory.
As part of the reduction step, we load `n` integers.
In total, our version performs `4 * n` memory accesses. 
However, if we were to implement this algorithm as a for loop, we'd likely write something along the lines of:

```c++
float max_diff = 0;
for (int i = 0; i < a.size(); i++) {
  max_diff = std::max(max_diff, std::abs(a[i] - b[i]));
}
```

This raw loop only performs `2 * n` memory accesses.
This means that materialization of products in memory leads to 2x overhead in terms of memory accesses.
But how does this affect performance?
Like the majority of algorithms, our algorithm is memory bound meaning the ratio of communication (memory accesses) to computation is so large that the overall performance of the code will be limited by the theoretical peak memory performance of the hardware.  As a programmer, this means we will focus performance optimizations on memory usage and data movement specifically.
Since our algorithm is memory bound, a two-fold reduction in amount of memory accesses should result in about two-fold speedup. 
Besides performance implications, the absolute differences that we compute occupy space in memory. 
If our vectors were large enough, there might be no space left in GPU memory for them and the temporary space we use to compute and save the differences.

## Iterators
Fortunately, there's a way to avoid materializing these differences in memory and address these issues.
The workaround consists of using iterators. 
Using an iterator can be thought of as a generalization of using a pointer: 
* A pointer, `int* pointer`, points to a sequence of integers in memory. 
* You can dereference a pointer to get access to the integer it currently points to.
* You can advance pointer with `pointer++` to make it point to the next element in the sequence.

The following code demonstrates using a pointer to access data in an array.

In [ ]:
%%writefile Sources/pointer.cpp
#include "ach.h"

int main()
{
    std::array<int, 3> a{ 0, 1, 2 };

    int *pointer = a.data();

    std::printf("pointer[0]: %d\n", pointer[0]); // prints 0
    std::printf("pointer[1]: %d\n", pointer[1]); // prints 1
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/pointer.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Simple Counting Iterator

C++ allows operator overloading. 
This means that we can define what operators such as `*` or `++` do.
The concept of an iterator builds on top of this idea.
With this, we don't even need an underlying container.
Here's an example of how we can create an infinite sequence without allocating a single byte.  Note the redefinition of the square brackets `[]` operator.

In [ ]:
%%writefile Sources/counting.cpp
#include "ach.h"

struct counting_iterator
{
  int operator[](int i)
  {
    return i;
  }
};

int main()
{
  counting_iterator it;

  std::printf("it[0]: %d\n", it[0]); // prints 0
  std::printf("it[1]: %d\n", it[1]); // prints 1
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/counting.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Simple Transform Iterator

Below we again redefine the square brackets `[]` operator, but this time instead of simple counting, we multiple each input value times 2.  This is an example of applying a simple function to the input before returning a value.

In [ ]:
%%writefile Sources/transform.cpp
#include "ach.h"

struct transform_iterator
{
  int *a;

  int operator[](int i)
  {
    return a[i] * 2;
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };

  transform_iterator it{a.data()};

  std::printf("it[0]: %d\n", it[0]); // prints 0 (0 * 2)
  std::printf("it[1]: %d\n", it[1]); // prints 2 (1 * 2)
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Simple Zip Iterator

We can continue to redefine the square brackets `[]` operator, to combine multiple sequences.  The zip iterator below takes two arrays and combines them into a sequence of tuples.

In [ ]:
%%writefile Sources/zip.cpp
#include "ach.h"

struct zip_iterator
{
  int *a;
  int *b;

  std::tuple<int, int> operator[](int i)
  {
    return {a[i], b[i]};
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  std::array<int, 3> b{ 5, 4, 2 };

  zip_iterator it{a.data(), b.data()};

  std::printf("it[0]: (%d, %d)\n", std::get<0>(it[0]), std::get<1>(it[0])); // prints (0, 5)
  std::printf("it[0]: (%d, %d)\n", std::get<0>(it[1]), std::get<1>(it[1])); // prints (1, 4)
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Combining Input Iterators

One very powerful feature of iterators is that you can combine them with each other.  If we think about our original code above where we computed the absolute value of the difference between each element in two arrays, you can see below that we can combine, or nest, the zip_iterator with the transform_iterator to first combine the two arrays `a` and `b` with zip, and then transform them via the transform iterator with our custom operation to compute the absolute value of the differences of each successive element in the original arrays `a` and `b`.

In [ ]:
%%writefile Sources/transform-zip.cpp
#include "ach.h"

struct zip_iterator
{
  int *a;
  int *b;

  std::tuple<int, int> operator[](int i)
  {
    return {a[i], b[i]};
  }
};

struct transform_iterator
{
  zip_iterator zip;

  int operator[](int i)
  {
    auto [a, b] = zip[i];
    return abs(a - b);
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  std::array<int, 3> b{ 5, 4, 2 };

  zip_iterator zip{a.data(), b.data()};
  transform_iterator it{zip};

  std::printf("it[0]: %d\n", it[0]); // prints 5
  std::printf("it[0]: %d\n", it[1]); // prints 3
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform-zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Transform Output Iterator

The concept of iterators is not limited to inputs alone.  With another level of indirection one can transform values that are written into a transform output iterator.  Note in the code below, both `=` and `[]` operators are being redefined.

In [ ]:
%%writefile Sources/transform-output.cpp
#include "ach.h"

struct wrapper
{
   int *ptr;

   void operator=(int value) {
      *ptr = value / 2;
   }
};

struct transform_output_iterator
{
  int *a;

  wrapper operator[](int i)
  {
    return {a + i};
  }
};

int main()
{
  std::array<int, 3> a{ 0, 1, 2 };
  transform_output_iterator it{a.data()};

  it[0] = 10;
  it[1] = 20;

  std::printf("a[0]: %d\n", a[0]); // prints 5
  std::printf("a[1]: %d\n", a[1]); // prints 10
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform-output.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

### Discard Iterator

In [ ]:
%%writefile Sources/discard.cpp
#include "ach.h"

struct wrapper
{
   void operator=(int value) {
      // discard value
   }
};

struct discard_iterator
{
  wrapper operator[](int i)
  {
    return {};
  }
};

int main()
{
  discard_iterator it{};

  it[0] = 10;
  it[1] = 20;
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/discard.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

## CUDA Fancy Iterators

CUDA Core Libraries provide a variety of iterators. 
Let's take a look at some of them as we try to improve the performance of our inner product implementation.
The first step is computing the absolute differences of corresponding vector components. 
To do that, we have to somehow make operator `*` return a pair of values, one taken from `a` and another taken from `b`.
This functionality is covered by `thrust::zip_iterator`.

In [ ]:
%%writefile Sources/zip.cpp
#include "ach.h"

int main()
{
    // allocate and initialize input vectors
    thrust::universal_vector<float> a{ 31, 22, 35 };
    thrust::universal_vector<float> b{ 25, 21, 27 };

    // zip two vectors into a single iterator
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());

    thrust::tuple<float, float> first = *zip;
    std::printf("first: (%g, %g)\n", thrust::get<0>(first), thrust::get<1>(first));

    zip++;

    thrust::tuple<float, float> second = *zip;
    std::printf("second: (%g, %g)\n", thrust::get<0>(second), thrust::get<1>(second));
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/zip.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

However, we don't need just pairs of vector components.
We need their absolute differences.
A `thrust::transform_iterator` allows us to attach a function to the dereferencing of an iterator. 
When combined with the zip iterator, it allows us to compute absolute differences without materializing them in memory.

In [ ]:
%%writefile Sources/transform.cpp
#include "ach.h"

int main()
{
    thrust::universal_vector<float> a{ 31, 22, 35 };
    thrust::universal_vector<float> b{ 25, 21, 27 };

    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });

    std::printf("first: %g\n", *transform); // absolute difference of `a[0] = 31` and `b[0] = 25`

    transform++;

    std::printf("second: %g\n", *transform); // absolute difference of `a[1] = 22` and `b[1] = 21`
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/transform.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

The only remaining part is computing a maximum value.
We already know how to do that using `thrust::reduce`.

Now, the code below is functionally equivalent to our starting code example at the top of this notebook.  Notice we have eliminated the need for the temporary array to store the differences.

In [ ]:
%%writefile Sources/optimized-max-diff.cpp
#include "ach.h"

float max_change(const thrust::universal_vector<float>& a,
                 const thrust::universal_vector<float>& b)
{
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });

    // compute max difference
    return thrust::reduce(thrust::device, transform, transform + a.size(), 0.0f, thrust::maximum<float>{});
}

int main()
{
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp[] = {{ 42, 24, 50 }, { 0, 0, 0}};
    auto transformation = [=] __host__ __device__ (float temp) { return temp + k * (ambient_temp - temp); };

    std::printf("step  max-change\n");
    for (int step = 0; step < 3; step++) {
        thrust::universal_vector<float> &current = temp[step % 2];
        thrust::universal_vector<float> &next = temp[(step + 1) % 2];

        thrust::transform(thrust::device, current.begin(), current.end(), next.begin(), transformation);
        std::printf("%d     %.2f\n", step, max_change(current, next));
    }
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/optimized-max-diff.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

Recall that this code is memory bound, so we'd expect that the elimination of unnecessary memory usage (in this case, temporary storage to hold the differences) should improve our performance. Let's evaluate performance of this implementation to see if it matches our intuition. 
To do that, we'll allocate much larger vectors.

In [ ]:
%%writefile Sources/naive-vs-iterators.cpp
#include "ach.h"

float naive_max_change(const thrust::universal_vector<float>& a,
                       const thrust::universal_vector<float>& b)
{
    thrust::universal_vector<float> diff(a.size());
    thrust::transform(thrust::device, a.begin(), a.end(), b.begin(), diff.begin(),
                      []__host__ __device__(float x, float y) {
                         return abs(x - y);
                      });
    return thrust::reduce(thrust::device, diff.begin(), diff.end(), 0.0f, thrust::maximum<float>{});
}

float max_change(const thrust::universal_vector<float>& a,
                 const thrust::universal_vector<float>& b)
{
    auto zip = thrust::make_zip_iterator(a.begin(), b.begin());
    auto transform = thrust::make_transform_iterator(zip, []__host__ __device__(thrust::tuple<float, float> t) {
        return abs(thrust::get<0>(t) - thrust::get<1>(t));
    });
    return thrust::reduce(thrust::device, transform, transform + a.size(), 0.0f, thrust::maximum<float>{});
}

int main()
{
    // allocate vectors containing 2^28 elements
    thrust::universal_vector<float> a(1 << 28);
    thrust::universal_vector<float> b(1 << 28);

    thrust::sequence(a.begin(), a.end());
    thrust::sequence(b.rbegin(), b.rend());

    auto start_naive = std::chrono::high_resolution_clock::now();
    naive_max_change(a, b);
    auto end_naive = std::chrono::high_resolution_clock::now();
    const double naive_duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_naive - start_naive).count();

    auto start = std::chrono::high_resolution_clock::now();
    max_change(a, b);
    auto end = std::chrono::high_resolution_clock::now();
    const double duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start).count();

    std::printf("iterators are %g times faster than naive approach\n", naive_duration / duration);
}

In [ ]:
!nvcc --extended-lambda -o /tmp/a.out Sources/naive-vs-iterators.cpp -x cu -arch=native # build executable
!/tmp/a.out # run executable

The resulting speedup exceeds our expectations because we included memory allocation in our measurements.

---
Proceed to [the exercise](01.03.02-Exercise-Computing-Variance.ipynb).

<img src="Images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">